In [1]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio
import iree.turbine.aot as aot

/home/gilro/ml_stuff/infra/iree/venv_online_everything/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

model.config.forced_decoder_ids = None

In [26]:
samples, sr = torchaudio.load("../data/gilro_test.wav")
input_ids = processor(
    samples[0][:], sampling_rate=sr, return_tensors="pt"
).input_features
predicted_ids = model.generate(input_ids)
transcription = processor.batch_decode(predicted_ids)
transcription

/home/gilro/ml_stuff/infra/iree/venv_online_everything/lib/python3.12/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/home/gilro/ml_stuff/infra/iree/venv_online_everything/lib/python3.12/site-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Pl

[' Hello, how are you? Are you here? Can you understand me?']

In [39]:
model = model.eval()


class Wrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return self.model.generate(x)
        # return self.model.model(
        #     x,
        # )


model_ = Wrapper(model).cpu().eval()

export_output = aot.export(model_, input_ids)


# model_(torch.randn((1,80,3000)))
model_.forward(input_ids)
# input_ids.shape

Unsupported: copy.deepcopy()
  Explanation: Dynamo does not support copy.deepcopy()
  Hint: Avoid calling copy.deepcopy()
  Hint: It may be possible to write Dynamo tracing rules for this code. Please report an issue to PyTorch if you encounter this graph break often and it is causing performance issues.

  Developer debug context: copy.deepcopy(UserDefinedObjectVariable(GenerationConfig))


from user code:
   File "/tmp/ipykernel_37830/2845661734.py", line 10, in forward
    return self.model.generate(x)
  File "/home/gilro/ml_stuff/infra/iree/venv_online_everything/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py", line 661, in generate
    generation_config, kwargs = self._prepare_generation_config(generation_config, **kwargs)
  File "/home/gilro/ml_stuff/infra/iree/venv_online_everything/lib/python3.12/site-packages/transformers/generation/utils.py", line 1754, in _prepare_generation_config
    generation_config = copy.deepcopy(generation_config)

Set 

In [ ]:
class NothingModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(100, 13)

    def forward(self, x):
        return torch.nn.ReLU()(self.fc(x))


model = NothingModel()
sample_inputs = torch.randn(100)
x = model(sample_inputs)

export_output = aot.export(model, sample_inputs)
export_output.save_mlir("converted_nothing_model.mlir")

Signature: export_output.save_mlir(file_path: Union[str, pathlib.Path])
Docstring:
Saves the current compilation IR to a path on disk.

Args:
    file_path: Path to save the file. If it has a ".mlirbc"
      extension, it will be saved as bytecode. Otherwise as
      text.
File:      ~/ml_stuff/infra/iree/venv_online_everything/lib/python3.12/site-packages/iree/turbine/aot/exporter.py
Type:      method